Implement a hashmap from scratch without any existing libraries in your preferred language. 

A hashmap should:

1. Be empty when initialized
2. Have the function `put(int key, int value)` which inserts a (key, value) pair into the hashmap. If the key already exists, update the corresponding value.
3. Have the function `get(int key)` which returns the value to which the specified key is mapped, or -1 if there’s no mapping for the key.
4. Have the function `remove(key)` which removes the key and its value if it exists in the map.

References: 
  - https://algs4.cs.princeton.edu/34hash/

### Expected usage
```
hashmap.put(key=3, value=100)
hashmap.get(key=3)
>> 100
hashmap.remove(key=3)
hashmap.get(key=3)
>> -1
```

### Solution sketch
1. **Hashing input to an array index for storage**. Since we are dealing only with integer keys, we can hash the input integer key with modulo M, where M is an arbitrarily chosen prime number signifying the size of the array, whose indices we are hashing the integer to e.g. if M is 11, and input integer is 23, 23 will be hashed to `23 % 11 = 1`, and hence 23 will be stored in the array at index 1
2. **Collision handling**. What do we do if another number hashes to the same index already occupied by 23? e.g. `12 % 11 = 1`, 12 also maps to index 1. We will represent this by storing these input numbers in a list of tuples at that array's index. Technically this should be implemented as a linked-list but we can keep things simple as Python lists can grow dynamically.

e.g.
```
[0] -> None
[1] -> [ (23, val_1), (12, val_2) ]
[2] -> None
[3] -> None
[4] -> None
[5] -> None
...
[10] -> None
```

In [2]:
# Here we choose an arbitrary hashmap size of 101 (array size is 101, so indices range from 0 to 100)
# Note that this number should be prime to reduce index collisions.
HASHMAP_SIZE = 101

In [33]:
class SimpleHashMap:
    def __init__(self, hashmap_size):
        self.hashmap_size = hashmap_size
        self.hashmap_arr = [ None for i in range(self.hashmap_size)]
        
    def put(self, key: int, value: int) -> None:
        index = key % self.hashmap_size
        
        # There are keys stored at this index.
        if self.hashmap_arr[index]:
            key_val_chain = self.hashmap_arr[index]
            
            # Iterate through each key to check if it already exists in key_val_chain. If it exists, overwrite the value
            found_key_idx = self._find_key_index_in_key_val_chain(key=key, key_val_chain=key_val_chain)
            
            # Found key-val tuple with input key
            if found_key_idx > -1:
                # Overwrite the old value with the new input value
                self.hashmap_arr[index][found_key_idx] = (key, value)
            else:
                # Append key and value as tuple in key_val_chain
                key_val_chain.append((key, value))
            
        # No keys stored at this index.
        else:
            # Make key_val_chain and store key, value as tuple.
            key_val_chain = [(key, value)]
            self.hashmap_arr[index] = key_val_chain
        
        return
    
    def get(self, key: int) -> int:
        index = key % self.hashmap_size
        
        # There is a key_val_chain stored at this index. 
        if self.hashmap_arr[index]:
            key_val_chain = self.hashmap_arr[index]
            # key_val_chain has length of 0 due to previous removals
            if len(key_val_chain) == 0:
                return -1
            
            # There are keys stored in the key_val_chain
            # Search for input key.
            found_key_idx = self._find_key_index_in_key_val_chain(key=key, key_val_chain=key_val_chain)
            
            # Found key-val tuple with input key
            if found_key_idx > -1:
                return key_val_chain[found_key_idx][1]
            # Searched through the whole key_val_chain, input key not found
            else:
                return -1
        
        # No keys stored at this index.
        else:    
            return -1
    
    def remove(self, key: int):
        index = key % self.hashmap_size
        
        # There are keys stored at this index. 
        if self.hashmap_arr[index]:
            # Search for input key.
            key_val_chain = self.hashmap_arr[index]
            
            # Iterate through each key to check if it already exists in key_val_chain. If it exists, overwrite the value
            found_key_idx = self._find_key_index_in_key_val_chain(key=key, key_val_chain=key_val_chain)
            
            # Found key-val tuple with input key
            if found_key_idx > -1:
                # Delete key-val tuple in key_val_chain at found_key_idx
                del self.hashmap_arr[index][found_key_idx]
    
    def _find_key_index_in_key_val_chain(self, key: int, key_val_chain: list) -> int:
        found_key_val_chain_idx = -1
        for key_val_idx, key_val_tuple in enumerate(key_val_chain):
            _key, _val = key_val_tuple
            # Found input key in key_val_chain, need to overwrite the old value with the new input value
            if _key == key:
                found_key_val_chain_idx = key_val_idx
                break
                
        return found_key_val_chain_idx
    
    def __str__(self):
        output_str = "{"
        for key_val_chain in self.hashmap_arr:
            if key_val_chain:
                for key_val_tuple in key_val_chain:
                    output_str += f"\n\t {key_val_tuple[0]} : {key_val_tuple[1]},"
                
        output_str += "\n} \n"
        return output_str

In [34]:
hashmap = SimpleHashMap(hashmap_size=HASHMAP_SIZE)

In [35]:
print(hashmap)

{
} 



In [36]:
hashmap.get(key=3)

-1

In [37]:
hashmap.put(key=3, value=100)

In [38]:
print(hashmap)

{
	 3 : 100,
} 



In [39]:
hashmap.get(key=3)

100

In [40]:
hashmap.put(key=3, value=9999)

In [41]:
hashmap.get(key=3)

9999

In [42]:
print(hashmap)

{
	 3 : 9999,
} 



In [43]:
hashmap.remove(key=3)

In [44]:
hashmap.get(key=3)

-1

In [45]:
print(hashmap)

{
} 



In [46]:
hashmap.get(key=10004314)

-1

In [47]:
hashmap.put(key=10004314, value=12121)

In [48]:
hashmap.get(key=10004314)

12121

In [49]:
print(hashmap)

{
	 10004314 : 12121,
} 



In [50]:
hashmap.remove(key=210)

In [51]:
hashmap.get(key=210)

-1

In [52]:
hashmap.remove(key=10004314)

In [53]:
hashmap.get(key=10004314)

-1

In [54]:
print(hashmap)

{
} 



In [55]:
hashmap.put(key=121212, value=329)
hashmap.put(key=20, value=0)
hashmap.put(key=1337, value=999)

In [56]:
print(str(hashmap))

{
	 121212 : 329,
	 20 : 0,
	 1337 : 999,
} 



In [57]:
# Keys 23 and 12 will collide as they both use index 1
hashmap.put(key=23, value=831)
hashmap.put(key=12, value=38)

In [58]:
print(str(hashmap))

{
	 121212 : 329,
	 12 : 38,
	 20 : 0,
	 23 : 831,
	 1337 : 999,
} 



In [59]:
hashmap.remove(key=12)

In [60]:
print(hashmap)

{
	 121212 : 329,
	 20 : 0,
	 23 : 831,
	 1337 : 999,
} 



In [61]:
hashmap.remove(key=23)

In [62]:
print(hashmap)

{
	 121212 : 329,
	 20 : 0,
	 1337 : 999,
} 

